In [3]:
import numpy as np
import torch
from torch_geometric.data import Data

torch.cuda.set_device(0)

# 引入attribute向量
attribute_embedding = np.load("generateData/Attribute_High_Embedding.npy")

# 引入category向量
category_embedding = np.load("generateData/Category_Embedding.npy")

# shop特征矩阵
shop_matrix = np.concatenate((attribute_embedding, category_embedding), axis=1)
padLine = np.zeros([1, attribute_embedding.shape[1]+category_embedding.shape[1]])
shop_matrix = np.insert(shop_matrix, 0, values=padLine, axis=0)    # pad 0
shop_matrix = torch.from_numpy(shop_matrix)

# user历史访问数据
user_history_dict = np.load('newData/UserHistoryData.npy', allow_pickle=True).item()

torch.Size([21041, 32])


In [4]:
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, degree

class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr='mean')  # "Add" aggregation.
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)

        # Step 3-5: Start propagating messages.
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)
    
    def message(self, x_j, edge_index, size):
        # x_j has shape [E, out_channels]

        # Step 3: Normalize node features.
        row, col = edge_index
        deg = degree(row, size[0], dtype=x_j.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        return norm.view(-1, 1) * x_j

    def update(self, aggr_out):
        # aggr_out has shape [N, out_channels]

        # Step 5: Return new node embeddings.
        return aggr_out

In [1]:
import torch.nn.functional as F
from torch import nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

embed_dim = shop_matrix.shape[1]
shop_size, shop_embed_size = shop_matrix.shape[0], shop_matrix.shape[1]

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.shop_size = shop_size
        self.shop_embed_size = shop_embed_size
        self.shop_embedding = nn.Embedding(self.shop_size, self.shop_embed_size, padding_idx=0)
        self.shop_embedding.weight.data.copy_(shop_matrix)
        self.shop_embedding.weight.requires_grad = False
        
        self.conv1 = GCNConv(embed_dim, embed_dim)
        self.bn = nn.BatchNorm1d(embed_dim)
        self.active = nn.ReLU()
        self.conv2 = GCNConv(embed_dim, embed_dim)

        self.lstm1 = nn.LSTM(input_size=embed_dim, hidden_size=embed_dim, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=embed_dim, hidden_size=embed_dim, batch_first=True)
        self.fc1 = nn.Sequential(
            nn.BatchNorm1d(2*embed_dim),
            nn.Linear(2*embed_dim, embed_dim),
            nn.ReLU()
        )
        self.fc2 = nn.Sequential(
            nn.BatchNorm1d(2*embed_dim),
            nn.Linear(2*embed_dim, embed_dim),
            nn.ReLU(),
            nn.BatchNorm1d(embed_dim),
            nn.Dropout(p=0.1),
            nn.Linear(embed_dim, 5)
        )
        
        # attention1参数
        self.w_omega = nn.Parameter(torch.Tensor(embed_dim, embed_dim))
        self.u_omega = nn.Parameter(torch.Tensor(embed_dim, 1))
        nn.init.uniform_(self.w_omega, -0.1, 0.1)
        nn.init.uniform_(self.u_omega, -0.1, 0.1)
        
        # attention2参数
        self.w_omega2 = nn.Parameter(torch.Tensor(embed_dim, embed_dim))
        self.u_omega2 = nn.Parameter(torch.Tensor(embed_dim, 1))
        nn.init.uniform_(self.w_omega2, -0.1, 0.1)
        nn.init.uniform_(self.u_omega2, -0.1, 0.1)
        
    def attention_net(self, x):   #x:[batch_size, seq_len, hidden_dim]
        u = torch.tanh(torch.matmul(x, self.w_omega))          #[batch, seq_len, hidden_dim]
        att = torch.matmul(u, self.u_omega)                           #[batch, seq_len, 1]
        att_score = F.softmax(att, dim=1)
        scored_x = x * att_score                           #[batch, seq_len, hidden_dim]
        feat = torch.sum(scored_x, dim=1)          #[batch, hidden_dim]
        return feat
        
    def attention_net2(self, x):   
        u = torch.tanh(torch.matmul(x, self.w_omega2))
        att = torch.matmul(u, self.u_omega2)
        att_score = F.softmax(att, dim=1)
        scored_x = x * att_score
        feat = torch.sum(scored_x, dim=1) 
        return feat

    def forward(self, shop_idxs, edge_index, userId, poiId, userHistory, userHistoryLength): 
        x = self.shop_embedding(shop_idxs)
        x = self.bn(x)
        x = self.conv1(x, edge_index)
        x = self.active(x)
        x = self.conv2(x, edge_index)
        x = self.active(x)
        region_embedding = nn.Embedding(self.shop_size, self.shop_embed_size).cuda()
        region_embedding.weight.data.copy_(x)
        
        currentPOI = self.shop_embedding(poiId)
        # 不等长lstm的batch输入处理，batch按照长度由大到小排序
        userHistory_lengths, idx = userHistoryLength.sort(0, descending=True)
        _, un_idx = torch.sort(idx, dim=0)
        userHistory = userHistory[idx]
        userHistory_POI = self.shop_embedding(userHistory)
        print("userHistory_POI")
        print(userHistory_POI)
        userHistory_POI_packed_input = pack_padded_sequence(input=userHistory_POI, lengths=userHistory_lengths, batch_first=True)
        # hidden_state集合；userHistory_POI_packed_out是[batch_size*seq_len,embed_dim]的tensor;_[0]是代表每个seq最后一个hiddenState构成的tensor,[1, batch_size, embedding]
        userHistory_POI_packed_out, (h_n, c_n) = self.lstm1(userHistory_POI_packed_input)
        # userHistory_POI_packed_out解压，仍按照长度由大到小排序, [batch_size,seq_len, embed_dim];_是一个tensor，每个元素代表每个seq的真实长度
        userHistory_POI_out, _ = pad_packed_sequence(userHistory_POI_packed_out, batch_first=True)
        # userHistory_POI_out恢复正常顺序，[batch_size,seq_len, embed_dim]，其后有补0
        userHistory_POI_out = torch.index_select(userHistory_POI_out, 0, un_idx)
        print("userHistory_POI_out")
        print(userHistory_POI_out)
        # 使用attention机制对seq的所有hidden_state加权, long_userPref_POI[batch_size, embed_dim]
        userAvePOIPref_attn = self.attention_net(userHistory_POI_out)
        userAvePOIPref = torch.cat([userAvePOIPref_attn, currentPOI], -1)
        # 每个seq的最后一个hidden_state构成的集合,short_userPref_POI[batch_size, embed_dim]
        userShortPOIPref = torch.index_select(h_n, 1, un_idx)
        userShortPOIPref = torch.squeeze(userShortPOIPref, 0)
        # userPref1 = torch.cat([userShortPOIPref, currentPOI], -1)
        
        currentPOIRegion = region_embedding(poiId)
        userHistory_Region = region_embedding(userHistory)
        userHistory_Region_packed_input = pack_padded_sequence(input=userHistory_Region, lengths=userHistory_lengths, batch_first=True)
        userHistory_Region_packed_out, (h_nr, c_nr) = self.lstm2(userHistory_Region_packed_input)
        userHistory_Region_out, _ = pad_packed_sequence(userHistory_Region_packed_out, batch_first=True)
        userHistory_Region_out = torch.index_select(userHistory_Region_out, 0, un_idx)
        userAveRegionPref_attn = self.attention_net2(userHistory_Region_out)
        # userLongRegionPref = torch.cat([userLongRegionPref_attn, currentPOI], -1)
        userShortRegionPref = torch.index_select(h_nr, 1, un_idx)
        userShortRegionPref = torch.squeeze(userShortRegionPref, 0)

        output = self.fc2(userAvePOIPref)
        return output

NameError: name 'shop_matrix' is not defined

In [ ]:
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, weight_decay=opt.weight_decay)